# Import Libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn import metrics
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Reading the data
data = pd.read_excel('../data/responses_data.xlsx')

In [3]:
# Checking the head of the data
data.head()

,description,answer_category_num,question_id_id
0,وزير الخارجية اللبناني جبران باسيل قال في سلسل...,Religious affiliation,1
1,وزير الخارجية اللبناني جبران باسيل قال في سلسل...,Religious affiliation,1
2,وزير الخارجية اللبناني جبران باسيل قال في سلسل...,Religious affiliation,1
3,وزير الخارجية اللبناني جبران باسيل قال في سلسل...,Racist,1
4,سورية بلد الحضارات تربطها بعلية او بحيوان,Violent,2


In [4]:
# The shape of the data
data.shape

(4825, 3)

In [5]:
data.description = data.description.apply(str.strip)

In [6]:
# The columns of the data
data.columns

Index(['description', 'answer_category_num', 'question_id_id'], dtype='object')

In [7]:
data.description.duplicated().sum()

1737

In [8]:
clean_data = data.copy()
clean_data  = clean_data[['description', 'answer_category_num']]
clean_data.head(2)

,description,answer_category_num
0,وزير الخارجية اللبناني جبران باسيل قال في سلسل...,Religious affiliation
1,وزير الخارجية اللبناني جبران باسيل قال في سلسل...,Religious affiliation


In [9]:
clean_data.description.value_counts()

لما العاهرة تاضر بالعفة                                                      9
كلامه صحيح                                                                   9
من يومهن الشيعة كيوت و اوبن مايندد                                           8
ما خصك ما دخلك افرنقع                                                        7
مبين ادي انت معفن ووسخ نظف حالك ولسانك افضل من انك تنشر وساختك عند النظاف    7
                                                                            ..
لا بس الوطن بدو تكنيس من ل متلك                                              1
يجب أن تعمم صحيح                                                             1
تحياتي للشعب العريق من الأردن 🇯🇴🇵🇸                                           1
اردوغان اكل هوا                                                              1
انا مين انت بدك تناقش فكرة ولا شخص فعلا انك قواتجي بهيم                      1
Name: description, Length: 3088, dtype: int64

In [10]:
sum(clean_data.description.value_counts() > 1)

1138

In [11]:
# removing duplicated descrptions/ comments
clean_data.drop_duplicates(subset='description', inplace=True)
clean_data.reset_index(drop=True, inplace=True)

In [12]:
clean_data

,description,answer_category_num
0,وزير الخارجية اللبناني جبران باسيل قال في سلسل...,Religious affiliation
1,سورية بلد الحضارات تربطها بعلية او بحيوان,Violent
2,تقتلون وسام الحسن وتترحموعلية من أي أصناف المخ...,Racist
3,معك خبر انو بلدة قطر متل ما سميتا مساحتها اكبر...,Normal
4,للامانه قوت الموسم اللي طاف كان هوا بس متحمس ح...,Normal
...,...,...
3083,من مبارح عم تتلج بلاريس من وين جابت الشمس,Normal
3084,العمى بقلبك شو مهضوم,Normal
3085,قديمة,Mockery
3086,الله يحميك يا بطل,Normal


In [13]:
# checking how many comments have different votes
for i, row in clean_data.iterrows():

    comment = row.description

    # getting the dataframe for that comment
    temp_df = data[data.description == comment]
    
    # how many unique answers that comment has
    n_ = temp_df.answer_category_num.nunique()

    # changing the answer_category_num to the most frequent one
    if n_ >= 2:
        most_voted_label = temp_df.answer_category_num.value_counts().index[0]
        clean_data.at[i, 'answer_category_num'] = most_voted_label

In [14]:
clean_data

,description,answer_category_num
0,وزير الخارجية اللبناني جبران باسيل قال في سلسل...,Religious affiliation
1,سورية بلد الحضارات تربطها بعلية او بحيوان,Violent
2,تقتلون وسام الحسن وتترحموعلية من أي أصناف المخ...,Racist
3,معك خبر انو بلدة قطر متل ما سميتا مساحتها اكبر...,Normal
4,للامانه قوت الموسم اللي طاف كان هوا بس متحمس ح...,Normal
...,...,...
3083,من مبارح عم تتلج بلاريس من وين جابت الشمس,Normal
3084,العمى بقلبك شو مهضوم,Normal
3085,قديمة,Mockery
3086,الله يحميك يا بطل,Normal


In [15]:
# Generated Class
clean_data.answer_category_num.value_counts()

Normal                   776
Mockery                  755
Violent                  571
Religious affiliation    368
Racist                   323
Sexual harrasment        295
Name: answer_category_num, dtype: int64

In [16]:
# Checking the info
clean_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3088 entries, 0 to 3087
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   description          3088 non-null   object
 1   answer_category_num  3088 non-null   object
dtypes: object(2)
memory usage: 48.4+ KB


In [17]:
# Removing stop words for each tweet
clean_data['tweet_no_stopwords'] = 'x'
stop_words = set(stopwords.words('arabic')) 
for count, tweet in enumerate(clean_data.description):
    word_tokens = word_tokenize(tweet)
    filtered_tweet = []
    for word in word_tokens:
        if word not in stop_words:
            filtered_tweet.append(word)
    joined_filtered_tweet = " ".join(filtered_tweet)
    clean_data.tweet_no_stopwords[count] = joined_filtered_tweet

In [18]:
clean_data.head(2)

,description,answer_category_num,tweet_no_stopwords
0,وزير الخارجية اللبناني جبران باسيل قال في سلسل...,Religious affiliation,وزير الخارجية اللبناني جبران باسيل قال سلسلة ت...
1,سورية بلد الحضارات تربطها بعلية او بحيوان,Violent,سورية بلد الحضارات تربطها بعلية او بحيوان


# Training the Model

In [19]:
# y is Class which is dependent on X Tweet
X = clean_data['tweet_no_stopwords']
y = clean_data['answer_category_num']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

In [20]:
y_train.shape, y_test.shape

((2624,), (464,))

In [21]:
y_train.value_counts()

Normal                   677
Mockery                  643
Violent                  478
Religious affiliation    305
Racist                   276
Sexual harrasment        245
Name: answer_category_num, dtype: int64

In [22]:
y_test.value_counts()

Mockery                  112
Normal                    99
Violent                   93
Religious affiliation     63
Sexual harrasment         50
Racist                    47
Name: answer_category_num, dtype: int64

In [23]:
def show_results(y_true, y_pred):
    print("Confusion Matrix")
    print(metrics.confusion_matrix(y_true, y_pred))

    print("Classification Report")
    print(metrics.classification_report(y_true, y_pred, )) 

### SVC

In [24]:
# Using pipiles for machine learning flow

text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC()),])


text_clf.fit(X_train, y_train)  


predictions = text_clf.predict(X_test)

In [25]:
show_results(y_test, predictions)

Confusion Matrix
[[43 27  7  4  2 29]
 [22 49  4 10  4 10]
 [17 10  3  1  2 14]
 [12 28  4  6  4  9]
 [ 9 19  3  3  2 14]
 [29 19  3  5  3 34]]
Classification Report
                       precision    recall  f1-score   support

              Mockery       0.33      0.38      0.35       112
               Normal       0.32      0.49      0.39        99
               Racist       0.12      0.06      0.08        47
Religious affiliation       0.21      0.10      0.13        63
    Sexual harrasment       0.12      0.04      0.06        50
              Violent       0.31      0.37      0.33        93

             accuracy                           0.30       464
            macro avg       0.23      0.24      0.23       464
         weighted avg       0.26      0.30      0.27       464



### Naive Bayes

In [26]:
# Using pipiles for machine learning flow
text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', MultinomialNB()),])


text_clf.fit(X_train, y_train)  


predictions = text_clf.predict(X_test)

In [27]:
show_results(y_test, predictions)

Confusion Matrix
[[63 38  0  0  0 11]
 [25 72  0  0  0  2]
 [26 18  0  0  0  3]
 [13 44  0  0  0  6]
 [17 23  0  0  0 10]
 [39 30  0  0  0 24]]
Classification Report
                       precision    recall  f1-score   support

              Mockery       0.34      0.56      0.43       112
               Normal       0.32      0.73      0.44        99
               Racist       0.00      0.00      0.00        47
Religious affiliation       0.00      0.00      0.00        63
    Sexual harrasment       0.00      0.00      0.00        50
              Violent       0.43      0.26      0.32        93

             accuracy                           0.34       464
            macro avg       0.18      0.26      0.20       464
         weighted avg       0.24      0.34      0.26       464



In [28]:
tst =  "انت  واحد حقير وما بتستحي ا"

In [29]:
text_clf.predict([tst])[0]

'Mockery'

In [30]:
wrong = 0
mock_viol = 0
correct_classified = 0

mock_viol_list = ['Violent', 'Mockery']

for count, test_tweet in enumerate(X_test):
    
    y_test_answer = y_test.values[count]
    y_test_prediction = predictions[count]

    # getting the correct classified comments
    if y_test_answer == y_test_prediction:
        correct_classified += 1

    if y_test_answer != y_test_prediction:
        print("Tweet: {}".format(test_tweet))
        print("Real Prediction: {} ||  Model Prediction: {}\n".format(y_test_answer, y_test_prediction))

        if y_test_answer in mock_viol_list and y_test_prediction in mock_viol_list:
            mock_viol += 1
        wrong += 1

print("Correct Classified {}".format(correct_classified))        
print("Misclassified {}".format(wrong))
mock_viol

Tweet: وهاب الجحش ماع ماع ماع
Real Prediction: Racist ||  Model Prediction: Normal

Tweet: كول هوا فطس لملكك انشالله بلكي منخلص عدد اليهود الموجودين المدعيين انن مسلملين
Real Prediction: Violent ||  Model Prediction: Mockery

Tweet: غايظهن جبران باسيل انت القلب
Real Prediction: Sexual harrasment ||  Model Prediction: Normal

Tweet: عاهاتك ياوطن
Real Prediction: Mockery ||  Model Prediction: Normal

Tweet: جبران باسيل فاشل بالسياسة صبي مغرور يدعي انه ذكي لايفكر يقول مستواه وتصرفاتهفضيحة بحق لبنان
Real Prediction: Violent ||  Model Prediction: Mockery

Tweet: جبران باسيل ورع اللبنانيين يضحكون
Real Prediction: Religious affiliation ||  Model Prediction: Normal

Tweet: داعم للإرهاب مكان لبنان فقط وحضر لبنان حباً نكاية يشاركوا
Real Prediction: Violent ||  Model Prediction: Normal

Tweet: ولاك انت يمكن سوري او جربوع او حيوان كون اكيد انو الحريري بصرماينو داعس عليكم
Real Prediction: Racist ||  Model Prediction: Mockery

Tweet: بدنا ندعس بقلبكن ولي شعب وسخ مجرم متلكن متل نظامكن مجرمين روحي انق

50

In [31]:
X_test.iloc[0]

'وهاب الجحش ماع ماع ماع'

# Getting questions id's for correctly classified questions by model

In [32]:
list_of_correct_ids = []

for i, test_tweet in enumerate(X_test):
    y_test_answer = y_test.values[i]
    y_test_prediction = predictions[i]

    # getting the correct classified comments
    if y_test_answer == y_test_prediction:
        # getting the original tweet before doing any preprocessing ex BOW removal
        description = clean_data[clean_data['tweet_no_stopwords'] == X_test.iloc[i]]['description'].values[0]

        # getting the question id
        question_id_id = data[data['description'] == description]['question_id_id'].values[0]

        list_of_correct_ids.append(question_id_id)

# Saving the Model

In [27]:
import pickle
# save the model to disk
filename = 'finalized_model_SVC.sav'
pickle.dump(text_clf, open(filename, 'wb'))

In [74]:
# some time later...
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.8393162393162393
